In [1]:
import numpy as np #importing needed libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import winsound as audio
import time
import os
import random as rand
from numpy.random import uniform
from random import choice
from os.path import exists
from openpyxl import Workbook
from openpyxl import load_workbook  
from pathlib import Path

In [ ]:
#######RANDOM DOTS####### Written by Lorenzo Reverberi, modified by Maarten Janzen

def rand_dot():
   
    baselines = [10, 30, 100, 300, 1000, 3000]                      #spaced by logx = 0.5
    differences = [0.0, 0.01, 0.03, 0.1, 0.33, 1.00]

    for b in baselines:
        for d in differences:
            
            n_pts = b # the base number of points you want
            delta_n = d # the variation in number around the baseline. 
                        # Note that as I'm writing it now, sometimes we get the
                        # same number of points (maybe it's a good idea to have that,
                        # guess it could help to estimate the intrinsic uncertainties
                        # in our data), but only 20% of the times.

            ### some figure settings
            fig_sz = 10 # figure size
            pt_sz = 15 # controls the point size
            ln_sz = 1 # circle line size
            circ_radius = 1.02 # > 1 so that points don't land on the edge. Might need a bigger value if pt_sz is bigger

            r = [-1,1]

            for num in r:

                fig, axs = plt.subplots(1, 2, figsize=(fig_sz,fig_sz))  #create 2 plots/figures
                ns = [n_pts, n_pts + num * (n_pts * delta_n)]           #assign number of dots for each figure
        
                for i in 0,1:

                    rs = np.sqrt(uniform(size = int(ns[i]))) 
                    thetas = 2. * np.pi * uniform(size = int(ns[i]))

                    xs = rs * np.cos(thetas)
                    ys = rs * np.sin(thetas)

                    axs[i].scatter(xs, ys, s=pt_sz)                     #create images
                    axs[i].add_patch(plt.Circle((0, 0), circ_radius, color='black', fill=False, lw=ln_sz)) 
                    axs[i].set_aspect('equal', 'box')
                    axs[i].axis('off')
                    axs[i].set_title('N_pts_' + str(i + 1) + ' = ' + str(ns[i]) + ' ~ ' +  str(delta_n * 100) + '%') # Delete this for images used in the survey
                    
                    print('N_pts_' + str(i + 1) + ' = ' + str(ns[i]) + ' ~ ' +  d + '%')

            ### Uncomment if you need to save this:
            title = 'N_pts_1-N_pts_2= ' + str(ns[0]) + '-' + str(ns[1]) + 'diff=' + str(d)
            
            while exists(title + '.pdf'):                               #making sure the files are saved without duplicate titles 
            
                inp = input('a file with the title ' + title + ' already exists.')
                title  = title + '_' + inp
                
            plt.savefig(title + '.jpg', bbox_inches = 'tight')          #save figure

    plt.show()

In [2]:
#######RANDOM CONTRAST####### Written by Lorenzo Reverberi, modified by Maarten Janzen

def rand_contrast():

    baselines = [0, 250, 500, 750, 1000]                                #fixed baseline colours
    differences = [-20, -15, -10, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20] 
    sh = False
   
    for b in baselines:                                                 #for ever baseline cycle through all possible differences 
        for d in differences:

            try:       
                                                                        #try loop as an attempt to prevent the code to generate images that are 
                                                                        #not possible (e.g baseline 1 + difference of 20)
                fig, ax = plt.subplots(nrows=1, ncols=1)

                delta_grey = d
                base_grey = b

                gr = base_grey
                gr_stim = base_grey + delta_grey
                gr_final = gr/1000
                gr_stim_final = gr_stim/1000

                ax.plot(0,0)
                ax.set_xbound(0, 10)
                ax.set_ybound(0, 10)
                ax.set_aspect('equal', 'box')
                ax.add_patch(patches.Rectangle((0,0), 10, 10, linewidth=1, facecolor=(gr_final,gr_final,gr_final)))
                ax.add_patch(patches.Rectangle((2.5,2.5), 5, 5, linewidth=1, facecolor=(gr_stim_final, gr_stim_final, gr_stim_final)))
                ax.axis('off')
                ax.set_title('Difference = ' + str(delta_grey)) # Delete this if don't want the titles

            except:

                plt.clf()                                               #if an exception occurs, don't show the figure (figure can't exist)

            title = 'gr-gr_stim-diff = ' + str(round(gr_final, 2)) + '-' + str(round(gr_stim_final, 2)) + '-' + str(round(abs(gr_final - gr_stim_final), 4))
        
            while exists(title + '.pdf'):                               #making sure the files are saved without duplicate titles
            
                inp = input('a file with the title ' + title + ' already exists.')
                title  = title + '_' + inp
                
            plt.savefig(title + '.jpg', bbox_inches = 'tight')          #save figure


In [ ]:
def volume_test(min, max): #written by Maarten Janzen

    print('adjust the volume of the system for low and high pitched sounds')
    
    t = False
    
    while t == False:

        audio.Beep(min, 1500)                                           #listen to the maximum and minimum frequency out of the tested range to make sure 
        time.sleep(0.5)                                                 #it won't be uncomfortable. can be played again to allow for volume adjustment
        audio.Beep(max, 1500)

        if input('play again? (y/n)') == 'y':
            t = False
        else:
            t = True

In [ ]:
#######RANDOM PITCH WITH BASELINE ~~VOLUME ADJUSTMENT~~######## Written by Lorenzo Reverberi, modified by Maarten Janzen

def pitch_base():

    name = input('name of file/participantcode')                        #getting initial file name to save the data to
    age = input('age')
    freq_title = name + '_' + age

    while exists(freq_title + '.txt'):                                  #if a file with this name exists, change the name
      
        inp = input('a file with the title ' + freq_title + ' already exists.')
        freq_title  = freq_title + '_' + inp
    
    file = open(freq_title + '.txt', 'a')                               #create the file

    baselines_low = [50, 100, 200, 500]                                 #baselines divided into seperate catagories to enable volume adjustment during the experiment
    baselines_high = [1000, 2000] 
    percent_diff  = [-0.10, -0.05, -0.02, -0.01, -0.005, -0.002, -0.001, 0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.10]
    
    print('test the volume for the lower frequencies')
    volume_test(50, 500)                                                #test the volume

    for bas in baselines_low:                                           #generate a tone for each baseline that plays until all differences have been tested, then move to next baseline
        for d in percent_diff:

            fr = bas
            fr_stim = bas + d * bas  

            audio.Beep(int(fr), 1500)                                   #generate first frequency
            time.sleep(0.5)                                             #wait 500 miliseconds
            audio.Beep(int(fr_stim), 1500)                              #generate second frequency
 

            ans = input('higher, lower or the same?(h/l/s)')            #ask for answer and write data to file
            file.write('freq_1 = ' + str(round(fr, 2)) + '\t|\tfreq_2 = ' + str(round(fr_stim, 2)) + '\t|\t' + ' delta = \t' + str(round((fr-fr_stim), 4)) + '\t\t\t|\tanswer ' + ans + '\n')
            
            ###example of generated data --> freq_1 = 50	|	freq_2 = 45.0	|	 delta = 	5.0			|	answer s### 

    print('test the volume for the higher frequencies')
    volume_test(1000, 2000)

    for base in baselines_high:
        for d in percent_diff:

            fr = base
            fr_stim = base + d * bas  

            audio.Beep(int(fr), 1500)
            time.sleep(0.5)
            audio.Beep(int(fr_stim), 1500)

            ans = input('higher, lower or the same?(h/l/s)')
            file.write('freq_1 = ' + str(round(fr, 2)) + '\t|\tfreq_2 = ' + str(round(fr_stim, 2)) + '\t|\t' + ' delta = \t' + str(round((fr - fr_stim), 4)) + '\t\t\t|\tanswer ' + str(ans) + '\n')

    file.close()                                                         #close the file
               
               

In [ ]:
#######PERPARE DATA######## Written by Maarten Janzen

def data_prep(filename):

    i = 0
    new_baseline = ''

    baselines = [50, 100, 200, 500, 1000, 2000]                         #use same baselines and differences as for the pitch experiment (only experiment with data saved to text file)
    percent_diff  = [-0.10, -0.05, -0.02, -0.01, -0.005, -0.002, -0.001, 0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.10]

    wb = Workbook()                                                     #create workbook in excel
    
    for b in baselines:                                                 #loop to create a sheet in the workbook for every baseline
                                                                        
        ws = wb.create_sheet()
        ws.title = str(b) + 'Hz'

    for files in os.listdir(Path('results new/')):                      #cycle through all the files generate from the experiment 

        f = open(Path('results new/' + files), 'r')                     #for every file, open the file and look through all lines

        for line in f:

            i += 1
                
            nlist = line.split('|')                                     #split each line in a list
            f1list = nlist[0]                                           #assign each element in the list to the corresponding data point
            f2list = nlist[1]
            dlist = nlist[2]
            an = nlist[3]

            new_baseline = f1list[f1list.find('=') + 2:f1list.find('/t')].strip()   #get the numbers that represent the data  
            new_freq = f2list[f2list.find('=') + 2:f2list.find('=') + 8].strip()
            delta = dlist[dlist.find('=') + 3:dlist.find('=') + 8].strip()

            if not ws == wb[str(new_baseline) + 'Hz']:                  #move to correct sheet in the workbook (corresponding to baseline frequency)
                
                ws = wb[str(new_baseline) + 'Hz']
                i = ws.max_row + 1                                      #make sure the data that might already be there is not overwritten by new data

            a = an[an.find(' '): an.find(' ') + 2].strip()
                   
            ws['A' + str(i)] = float(new_baseline)                      #write the data to the excel file in dedicated columns as numbers (float, only if necessary)
            ws['B' + str(i)] = float(new_freq)
            ws['C' + str(i)] = float(delta)
            ws['D' + str(i)] = a  
            
            if (float(new_baseline) < float(new_freq) and a ==  'h') or (float(new_baseline) == float(new_freq) and a ==  's') or (float(new_baseline) > float(new_freq) and a ==  'l'):
                                                                        #check whether or not the given answer was correct
                ws['E' + str(i)] = 'correct'                            #if it was, write it to the file in a new column
                
            else:
                
                ws['E' + str(i)] = 'incorrect'              
    
    wb.save(filename + '.xlsx')                                         #save the excel file
    wb.close()                                                          #close the excel file,         
    f.close()                                                           #close the read file before opening a new one

    #this only generates 1 excel file per run

In [ ]:
#######ANALYSE THE DATA######## Written by Maarten Janzen

def analysis(filename): 

    load_wb = load_workbook(filename + '.xlsx')                         #load same excel file as created in data_prep()
    wb = Workbook()                                                     #create a new excel file
    correct = 0
    incorrect = 0  

    baselines = [50, 100, 200, 500, 1000, 2000]                         #use same baselines and differences as for the pitch experiment (only experiment with data saved to text file)
    percent_diff  = [-0.10, -0.05, -0.02, -0.01, -0.005, -0.002, -0.001, 0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.10]    

    for name in load_wb.sheetnames:                                     #for every sheet in the old file, create a sheet in the new file with the same name

        load_ws = load_wb[name]                                         
        num_of_attampts_per_diff = int((load_ws.max_row - 1) / len(percent_diff))   #determine max number of attempts (participants) per difference in frequency 
        i = 0      #reset i for every baseline                                     #by dividing the maximum number of rows in the old file by the total amount of 
                                                                                    #difference frequencies
        ws = wb.create_sheet()
        ws.title = name

        for r in range(2, load_ws.max_row, len(os.listdir(Path('results new/')))):  #per baseline (first for loop), find the different baselines frequencies and check how 
                                                                                    #many participants got it right (third for loop). every baseline frequency has evenly spaced 
            correct = 0                                                             #start and end points per difference frequency. this space is the same number of rows as there were participants
            incorrect = 0
            i += 1                                                                  #make i (row coordinate for new file) increase by 1 for every difference

            for ra in range(0, num_of_attampts_per_diff):                           #check for number of right answers per difference frequency

                if load_ws['E' + str(r + ra)].value == 'correct': correct += 1      #if correct add 1 to the correct variable
                if load_ws['E' + str(r + ra)].value == 'incorrect': incorrect += 1  #if incorrect add 1 to the incorrect variable

            percent_correct = round((correct / num_of_attampts_per_diff) * 100, 2)  #determine percentage of correct answers per difference frequency per baseline frequency

            if not ws.title == load_ws.title:                                       #make sure the right sheet is selected
                
                ws = wb[name]
            
            ws['A' + str(i)] = load_ws['C' + str(r)].value                          #write difference in Hz
            ws['B' + str(i)] = round((correct / num_of_attampts_per_diff) * 100, 2) #write percentage of correct answers

    wb.save(filename  + '_percent.xlsx')                                            #save the file
    wb.close()                                                                      #close the file